In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
!pip install sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.0 MB/s eta 0:00:00


# Load Data

In [ ]:
df_pantai = pd.read_csv('https://drive.google.com/uc?id=1GTNiDZzzVbbyn9aObjsXTeGqDBBwQTpo')
df_review = pd.read_csv('https://drive.google.com/uc?export=download&id=1vrypAVzgXunzg7RLSOH1lISbp7Gmel3f')

In [ ]:
df_pantai.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2392 entries, 0 to 2391
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   place_id         2392 non-null   object 
 1   name             2392 non-null   object 
 2   description      2392 non-null   object 
 3   reviews          2392 non-null   int64  
 4   rating           2392 non-null   float64
 5   featured_image   2391 non-null   object 
 6   address          2381 non-null   object 
 7   review_keywords  2392 non-null   object 
 8   link             2392 non-null   object 
 9   coordinates      2392 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 187.0+ KB


In [ ]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300949 entries, 0 to 300948
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   place_id       300949 non-null  object
 1   place_name     300949 non-null  object
 2   text_stopword  300949 non-null  object
dtypes: object(3)
memory usage: 6.9+ MB


In [ ]:
df_pantai.columns

Index(['place_id', 'name', 'description', 'reviews', 'rating',
       'featured_image', 'address', 'review_keywords', 'link', 'coordinates'],
      dtype='object')

In [ ]:
df_review.columns

Index(['place_id', 'place_name', 'text_stopword'], dtype='object')

# Natural Language Understanding + Content-Based Recommendation

In [ ]:
df_pantai.head()

,place_id,name,description,reviews,rating,featured_image,address,review_keywords,link,coordinates
0,timur_0001,Pantai Natsepa,"Jalur pasir putih dengan pemandian, banana boa...",2217,4.4,https://lh3.ggpht.com/p/AF1QipMYYpqAHoy1IcbdyU...,"Suli, Kec. Salahutu, Kabupaten Maluku Tengah, ...","rujak, pemandangan, buah, es kelapa muda, bana...",https://www.google.com/maps/place/Pantai+Natse...,"-3.6215465, 128.2921198"
1,timur_0002,Pantai Liang,-,1246,4.5,https://lh3.ggpht.com/p/AF1QipOQRzqn26_fRZauPI...,"Jalan Propinsi, Liang, Salahutu, Kabupaten Mal...","tempat wisata, pemandangan, rujak, banana boat...",https://www.google.com/maps/place/Pantai+Liang...,"-3.5049264, 128.3428253"
2,timur_0004,Pantai Ngurbloat Desa Ngilngof - Kei Islands,Hamparan pasir putih panjang berlatar pohon ke...,1070,4.7,https://lh3.ggpht.com/p/AF1QipNsrLGl1G85bTvrLW...,"Ngurbloat, Kabupaten Maluku Tenggara, Maluku","pasir, sunset, panjang, tepung, pemandangan, s...",https://www.google.com/maps/place/Pantai+Ngurb...,"-5.6624981, 132.63623959999998"
3,timur_0005,Pantai Pintu Kota Ambon,Tempat berenang tertutup di samping pantai ber...,1021,4.4,https://lh3.ggpht.com/p/AF1QipO5v7I-TAjeg39YGl...,"Nusaniwe, Kec. Nusaniwe, Kota Ambon, Maluku","pemandangan, ombak, berenang, tebing, kelapa, ...",https://www.google.com/maps/place/Pantai+Pintu...,"-3.7705365, 128.1524331"
4,timur_0006,Pantai Kota Jawa,-,660,4.4,https://lh3.ggpht.com/p/AF1QipN5SNMOuiDZAZORPr...,"Jl. Ir. M. Putuhena, Rumah Tiga, Kec. Tlk. Amb...","santai, sore, pemandangan, snack, pinggir pant...",https://www.google.com/maps/place/Pantai+Kota+...,"-3.6613789, 128.1790623"


In [ ]:
df_review.head()

,place_id,place_name,text_stopword
0,ChIJtwSbBE2LbC0RFPhn-0LftzQ,Pantai Liang,"['pantainya', 'bagus', 'pasir', 'putih', 'air'..."
1,ChIJtwSbBE2LbC0RFPhn-0LftzQ,Pantai Liang,"['pantainya', 'bersih', 'biaya', 'masuk', 'mah..."
2,ChIJtwSbBE2LbC0RFPhn-0LftzQ,Pantai Liang,"['pantainya', 'bagus', 'dijadikan', 'piknik', ..."
3,ChIJtwSbBE2LbC0RFPhn-0LftzQ,Pantai Liang,"['butuh', 'jam', 'pantai', 'liang', 'pantainya..."
4,ChIJtwSbBE2LbC0RFPhn-0LftzQ,Pantai Liang,"['tempatnya', 'indah', 'banget', 'alhamdulilla..."


In [ ]:
# prompt: buat df baru yang dimana menggabungkan text_stopword dari df_review dan dimasukan ke df_pantai pada kolom review_keywords	berdasarkan place_name

# Group reviews by place_name and join the text_stopword
review_keywords_df = df_review.groupby('place_name')['text_stopword'].apply(lambda x: ' '.join(x)).reset_index()
review_keywords_df.rename(columns={'text_stopword': 'review_keywords'}, inplace=True)

print(review_keywords_df.head())

                    place_name  \
0  101 Nusa Lima Beach & Resto   
1           Adhi Pradana Beach   
2               Agal Waterfall   
3       Agro Wisata Dream Land   
4            Air Terjun Bintan   

                                     review_keywords  
0  ['wisata', 'pemandangan', 'hamparan', 'laut', ...  
1  ['bagus', 'pantai', 'sayang', 'pengelolaan', '...  
2  ['air', 'terjun', 'agal', 'desa', 'marenteh', ...  
3  ['segi', 'jalan', 'lokasi', 'bnyak', 'jalan', ...  
4  ['salah', 'favorit', 'refreshing', 'gunung', '...  


In [ ]:
review_keywords_df.shape

(3041, 2)

In [ ]:
df_review.shape

(300949, 3)

In [ ]:
df_pantai.rename(columns={'name': 'place_name'}, inplace=True)

In [ ]:
# Merge the new column into df_pantai based on place_name
df_cbr = pd.merge(df_pantai, review_keywords_df, on='place_name', how='left')

In [ ]:
df_cbr

,place_id,place_name,description,reviews,rating,featured_image,address,review_keywords_x,link,coordinates,review_keywords_y
0,timur_0001,Pantai Natsepa,"Jalur pasir putih dengan pemandian, banana boa...",2217,4.4,https://lh3.ggpht.com/p/AF1QipMYYpqAHoy1IcbdyU...,"Suli, Kec. Salahutu, Kabupaten Maluku Tengah, ...","rujak, pemandangan, buah, es kelapa muda, bana...",https://www.google.com/maps/place/Pantai+Natse...,"-3.6215465, 128.2921198","['pantai', 'natsepa', 'ambon', 'wisata', 'meng..."
1,timur_0002,Pantai Liang,-,1246,4.5,https://lh3.ggpht.com/p/AF1QipOQRzqn26_fRZauPI...,"Jalan Propinsi, Liang, Salahutu, Kabupaten Mal...","tempat wisata, pemandangan, rujak, banana boat...",https://www.google.com/maps/place/Pantai+Liang...,"-3.5049264, 128.3428253","['pantainya', 'bagus', 'pasir', 'putih', 'air'..."
2,timur_0004,Pantai Ngurbloat Desa Ngilngof - Kei Islands,Hamparan pasir putih panjang berlatar pohon ke...,1070,4.7,https://lh3.ggpht.com/p/AF1QipNsrLGl1G85bTvrLW...,"Ngurbloat, Kabupaten Maluku Tenggara, Maluku","pasir, sunset, panjang, tepung, pemandangan, s...",https://www.google.com/maps/place/Pantai+Ngurb...,"-5.6624981, 132.63623959999998","['pantai', 'ngurbloat', 'terkenal', 'pasir', '..."
3,timur_0005,Pantai Pintu Kota Ambon,Tempat berenang tertutup di samping pantai ber...,1021,4.4,https://lh3.ggpht.com/p/AF1QipO5v7I-TAjeg39YGl...,"Nusaniwe, Kec. Nusaniwe, Kota Ambon, Maluku","pemandangan, ombak, berenang, tebing, kelapa, ...",https://www.google.com/maps/place/Pantai+Pintu...,"-3.7705365, 128.1524331","['dinamakan', 'pintu', 'kota', 'karang', 'teng..."
4,timur_0006,Pantai Kota Jawa,-,660,4.4,https://lh3.ggpht.com/p/AF1QipN5SNMOuiDZAZORPr...,"Jl. Ir. M. Putuhena, Rumah Tiga, Kec. Tlk. Amb...","santai, sore, pemandangan, snack, pinggir pant...",https://www.google.com/maps/place/Pantai+Kota+...,"-3.6613789, 128.1790623","['salah', 'spot', 'nongkrong', 'murah', 'meria..."
...,...,...,...,...,...,...,...,...,...,...,...
2387,tengah_2079,pantai Tanjung kelayang,-,12,4.8,https://lh3.ggpht.com/p/AC9h4nokxcrPUAUVdEaAfR...,"Keciput, Kabupaten Belitung, Kepulauan Bangka ...","pulau, pesisir",https://www.google.com/maps/place/pantai+Tanju...,"-2.5565735, 107.6683017","['cuaca', 'buruk', 'mesti', 'pasrah', 'deh', '..."
2388,tengah_2081,Pantai Family,-,302,4.3,https://lh3.ggpht.com/p/AC9h4nqym1AcQ6gJGrrCCT...,"Muara kandis, Kec. Linggo Sari Baganti, Kabupa...","pantainya, anak, parkir, rekreasi, biaya, sant...",https://www.google.com/maps/place/Pantai+Famil...,"-1.9128284, 100.85833509999999","['pantainya', 'ramai', 'bayar', 'parkir', 'rb'..."
2389,tengah_2083,Pantai Biru,-,40,4.4,https://lh3.ggpht.com/p/AC9h4np6EdUzRuVxnWzvkc...,"Muara Kandis, Punggasan, Kec. Linggo Sari Baga...",namanya,https://www.google.com/maps/place/Pantai+Biru/...,"-1.8989821, 100.8507066","['airnya', 'jernih', 'enak', 'main', 'air', 'b..."
2390,tengah_2084,Pantai Putra Deli,-,146,3.9,https://lh3.ggpht.com/p/AC9h4noBMlvxBxAEySyyGC...,"Kubah Sentang, Kec. Pantai Labu, Kabupaten Del...","parkir, pungli, laut, biaya, santai, pemandang...",https://www.google.com/maps/place/Pantai+Putra...,"3.6792145, 98.9163749","['jarak', 'tempuh', 'kota', 'medan', 'lokasi',..."


In [ ]:
df_cbr['review_keywords_y'] = df_cbr['review_keywords_y'].astype(str).str.replace('[\[\]\'\']', '', regex=True).str.replace(r',\s*', ', ', regex=True)
df_cbr

,place_id,place_name,description,reviews,rating,featured_image,address,review_keywords_x,link,coordinates,review_keywords_y
0,timur_0001,Pantai Natsepa,"Jalur pasir putih dengan pemandian, banana boa...",2217,4.4,https://lh3.ggpht.com/p/AF1QipMYYpqAHoy1IcbdyU...,"Suli, Kec. Salahutu, Kabupaten Maluku Tengah, ...","rujak, pemandangan, buah, es kelapa muda, bana...",https://www.google.com/maps/place/Pantai+Natse...,"-3.6215465, 128.2921198","pantai, natsepa, ambon, wisata, menginjakkan, ..."
1,timur_0002,Pantai Liang,-,1246,4.5,https://lh3.ggpht.com/p/AF1QipOQRzqn26_fRZauPI...,"Jalan Propinsi, Liang, Salahutu, Kabupaten Mal...","tempat wisata, pemandangan, rujak, banana boat...",https://www.google.com/maps/place/Pantai+Liang...,"-3.5049264, 128.3428253","pantainya, bagus, pasir, putih, air, tenang, s..."
2,timur_0004,Pantai Ngurbloat Desa Ngilngof - Kei Islands,Hamparan pasir putih panjang berlatar pohon ke...,1070,4.7,https://lh3.ggpht.com/p/AF1QipNsrLGl1G85bTvrLW...,"Ngurbloat, Kabupaten Maluku Tenggara, Maluku","pasir, sunset, panjang, tepung, pemandangan, s...",https://www.google.com/maps/place/Pantai+Ngurb...,"-5.6624981, 132.63623959999998","pantai, ngurbloat, terkenal, pasir, terhalus, ..."
3,timur_0005,Pantai Pintu Kota Ambon,Tempat berenang tertutup di samping pantai ber...,1021,4.4,https://lh3.ggpht.com/p/AF1QipO5v7I-TAjeg39YGl...,"Nusaniwe, Kec. Nusaniwe, Kota Ambon, Maluku","pemandangan, ombak, berenang, tebing, kelapa, ...",https://www.google.com/maps/place/Pantai+Pintu...,"-3.7705365, 128.1524331","dinamakan, pintu, kota, karang, tengahnya, ber..."
4,timur_0006,Pantai Kota Jawa,-,660,4.4,https://lh3.ggpht.com/p/AF1QipN5SNMOuiDZAZORPr...,"Jl. Ir. M. Putuhena, Rumah Tiga, Kec. Tlk. Amb...","santai, sore, pemandangan, snack, pinggir pant...",https://www.google.com/maps/place/Pantai+Kota+...,"-3.6613789, 128.1790623","salah, spot, nongkrong, murah, meriah, ambon, ..."
...,...,...,...,...,...,...,...,...,...,...,...
2387,tengah_2079,pantai Tanjung kelayang,-,12,4.8,https://lh3.ggpht.com/p/AC9h4nokxcrPUAUVdEaAfR...,"Keciput, Kabupaten Belitung, Kepulauan Bangka ...","pulau, pesisir",https://www.google.com/maps/place/pantai+Tanju...,"-2.5565735, 107.6683017","cuaca, buruk, mesti, pasrah, deh, rencana, tri..."
2388,tengah_2081,Pantai Family,-,302,4.3,https://lh3.ggpht.com/p/AC9h4nqym1AcQ6gJGrrCCT...,"Muara kandis, Kec. Linggo Sari Baganti, Kabupa...","pantainya, anak, parkir, rekreasi, biaya, sant...",https://www.google.com/maps/place/Pantai+Famil...,"-1.9128284, 100.85833509999999","pantainya, ramai, bayar, parkir, rb, bayar, ma..."
2389,tengah_2083,Pantai Biru,-,40,4.4,https://lh3.ggpht.com/p/AC9h4np6EdUzRuVxnWzvkc...,"Muara Kandis, Punggasan, Kec. Linggo Sari Baga...",namanya,https://www.google.com/maps/place/Pantai+Biru/...,"-1.8989821, 100.8507066","airnya, jernih, enak, main, air, bareng, kelua..."
2390,tengah_2084,Pantai Putra Deli,-,146,3.9,https://lh3.ggpht.com/p/AC9h4noBMlvxBxAEySyyGC...,"Kubah Sentang, Kec. Pantai Labu, Kabupaten Del...","parkir, pungli, laut, biaya, santai, pemandang...",https://www.google.com/maps/place/Pantai+Putra...,"3.6792145, 98.9163749","jarak, tempuh, kota, medan, lokasi, pantai, ja..."


In [ ]:
# Check if the columns exist before attempting to combine
if 'review_keywords_x' in df_cbr.columns and 'review_keywords_y' in df_cbr.columns:
    # Use fillna('') to treat missing values as empty strings for concatenation
    df_cbr['combined_review_keywords'] = df_cbr['review_keywords_x'].fillna('') + ' ' + df_cbr['review_keywords_y'].fillna('')

    # Optional: Clean up extra spaces caused by concatenation with empty strings
    df_cbr['combined_review_keywords'] = df_cbr['combined_review_keywords'].str.strip()
    df_cbr['combined_review_keywords'] = df_cbr['combined_review_keywords'].str.replace(r'\s+', ' ', regex=True)

    # Display the first few rows with the new column
    print(df_cbr[['place_name', 'review_keywords_x', 'review_keywords_y', 'combined_review_keywords']].head())
else:
    print("Columns 'review_keywords_x' or 'review_keywords_y' not found in the DataFrame.")

                                     place_name  \
0                                Pantai Natsepa   
1                                  Pantai Liang   
2  Pantai Ngurbloat Desa Ngilngof - Kei Islands   
3                       Pantai Pintu Kota Ambon   
4                              Pantai Kota Jawa   

                                   review_keywords_x  \
0  rujak, pemandangan, buah, es kelapa muda, bana...   
1  tempat wisata, pemandangan, rujak, banana boat...   
2  pasir, sunset, panjang, tepung, pemandangan, s...   
3  pemandangan, ombak, berenang, tebing, kelapa, ...   
4  santai, sore, pemandangan, snack, pinggir pant...   

                                   review_keywords_y  \
0  pantai, natsepa, ambon, wisata, menginjakkan, ...   
1  pantainya, bagus, pasir, putih, air, tenang, s...   
2  pantai, ngurbloat, terkenal, pasir, terhalus, ...   
3  dinamakan, pintu, kota, karang, tengahnya, ber...   
4  salah, spot, nongkrong, murah, meriah, ambon, ...   

                   

In [ ]:
df_cbr = df_cbr.drop(columns=['review_keywords_x', 'review_keywords_y'])
df_cbr

,place_id,place_name,description,reviews,rating,featured_image,address,link,coordinates,combined_review_keywords
0,timur_0001,Pantai Natsepa,"Jalur pasir putih dengan pemandian, banana boa...",2217,4.4,https://lh3.ggpht.com/p/AF1QipMYYpqAHoy1IcbdyU...,"Suli, Kec. Salahutu, Kabupaten Maluku Tengah, ...",https://www.google.com/maps/place/Pantai+Natse...,"-3.6215465, 128.2921198","rujak, pemandangan, buah, es kelapa muda, bana..."
1,timur_0002,Pantai Liang,-,1246,4.5,https://lh3.ggpht.com/p/AF1QipOQRzqn26_fRZauPI...,"Jalan Propinsi, Liang, Salahutu, Kabupaten Mal...",https://www.google.com/maps/place/Pantai+Liang...,"-3.5049264, 128.3428253","tempat wisata, pemandangan, rujak, banana boat..."
2,timur_0004,Pantai Ngurbloat Desa Ngilngof - Kei Islands,Hamparan pasir putih panjang berlatar pohon ke...,1070,4.7,https://lh3.ggpht.com/p/AF1QipNsrLGl1G85bTvrLW...,"Ngurbloat, Kabupaten Maluku Tenggara, Maluku",https://www.google.com/maps/place/Pantai+Ngurb...,"-5.6624981, 132.63623959999998","pasir, sunset, panjang, tepung, pemandangan, s..."
3,timur_0005,Pantai Pintu Kota Ambon,Tempat berenang tertutup di samping pantai ber...,1021,4.4,https://lh3.ggpht.com/p/AF1QipO5v7I-TAjeg39YGl...,"Nusaniwe, Kec. Nusaniwe, Kota Ambon, Maluku",https://www.google.com/maps/place/Pantai+Pintu...,"-3.7705365, 128.1524331","pemandangan, ombak, berenang, tebing, kelapa, ..."
4,timur_0006,Pantai Kota Jawa,-,660,4.4,https://lh3.ggpht.com/p/AF1QipN5SNMOuiDZAZORPr...,"Jl. Ir. M. Putuhena, Rumah Tiga, Kec. Tlk. Amb...",https://www.google.com/maps/place/Pantai+Kota+...,"-3.6613789, 128.1790623","santai, sore, pemandangan, snack, pinggir pant..."
...,...,...,...,...,...,...,...,...,...,...
2387,tengah_2079,pantai Tanjung kelayang,-,12,4.8,https://lh3.ggpht.com/p/AC9h4nokxcrPUAUVdEaAfR...,"Keciput, Kabupaten Belitung, Kepulauan Bangka ...",https://www.google.com/maps/place/pantai+Tanju...,"-2.5565735, 107.6683017","pulau, pesisir cuaca, buruk, mesti, pasrah, de..."
2388,tengah_2081,Pantai Family,-,302,4.3,https://lh3.ggpht.com/p/AC9h4nqym1AcQ6gJGrrCCT...,"Muara kandis, Kec. Linggo Sari Baganti, Kabupa...",https://www.google.com/maps/place/Pantai+Famil...,"-1.9128284, 100.85833509999999","pantainya, anak, parkir, rekreasi, biaya, sant..."
2389,tengah_2083,Pantai Biru,-,40,4.4,https://lh3.ggpht.com/p/AC9h4np6EdUzRuVxnWzvkc...,"Muara Kandis, Punggasan, Kec. Linggo Sari Baga...",https://www.google.com/maps/place/Pantai+Biru/...,"-1.8989821, 100.8507066","namanya airnya, jernih, enak, main, air, baren..."
2390,tengah_2084,Pantai Putra Deli,-,146,3.9,https://lh3.ggpht.com/p/AC9h4noBMlvxBxAEySyyGC...,"Kubah Sentang, Kec. Pantai Labu, Kabupaten Del...",https://www.google.com/maps/place/Pantai+Putra...,"3.6792145, 98.9163749","parkir, pungli, laut, biaya, santai, pemandang..."


In [ ]:
df_cbr = df_cbr.drop(columns=['reviews',	'rating', 'featured_image', 'link',	'coordinates', 'address'])
df_cbr

,place_id,place_name,description,combined_review_keywords
0,timur_0001,Pantai Natsepa,"Jalur pasir putih dengan pemandian, banana boa...","rujak, pemandangan, buah, es kelapa muda, bana..."
1,timur_0002,Pantai Liang,-,"tempat wisata, pemandangan, rujak, banana boat..."
2,timur_0004,Pantai Ngurbloat Desa Ngilngof - Kei Islands,Hamparan pasir putih panjang berlatar pohon ke...,"pasir, sunset, panjang, tepung, pemandangan, s..."
3,timur_0005,Pantai Pintu Kota Ambon,Tempat berenang tertutup di samping pantai ber...,"pemandangan, ombak, berenang, tebing, kelapa, ..."
4,timur_0006,Pantai Kota Jawa,-,"santai, sore, pemandangan, snack, pinggir pant..."
...,...,...,...,...
2387,tengah_2079,pantai Tanjung kelayang,-,"pulau, pesisir cuaca, buruk, mesti, pasrah, de..."
2388,tengah_2081,Pantai Family,-,"pantainya, anak, parkir, rekreasi, biaya, sant..."
2389,tengah_2083,Pantai Biru,-,"namanya airnya, jernih, enak, main, air, baren..."
2390,tengah_2084,Pantai Putra Deli,-,"parkir, pungli, laut, biaya, santai, pemandang..."


In [ ]:
df_cbr['combined_text'] = df_cbr['description'] + " " + df_cbr['combined_review_keywords']
df_cbr

,place_id,place_name,description,combined_review_keywords,combined_text
0,timur_0001,Pantai Natsepa,"Jalur pasir putih dengan pemandian, banana boa...","rujak, pemandangan, buah, es kelapa muda, bana...","Jalur pasir putih dengan pemandian, banana boa..."
1,timur_0002,Pantai Liang,-,"tempat wisata, pemandangan, rujak, banana boat...","- tempat wisata, pemandangan, rujak, banana bo..."
2,timur_0004,Pantai Ngurbloat Desa Ngilngof - Kei Islands,Hamparan pasir putih panjang berlatar pohon ke...,"pasir, sunset, panjang, tepung, pemandangan, s...",Hamparan pasir putih panjang berlatar pohon ke...
3,timur_0005,Pantai Pintu Kota Ambon,Tempat berenang tertutup di samping pantai ber...,"pemandangan, ombak, berenang, tebing, kelapa, ...",Tempat berenang tertutup di samping pantai ber...
4,timur_0006,Pantai Kota Jawa,-,"santai, sore, pemandangan, snack, pinggir pant...","- santai, sore, pemandangan, snack, pinggir pa..."
...,...,...,...,...,...
2387,tengah_2079,pantai Tanjung kelayang,-,"pulau, pesisir cuaca, buruk, mesti, pasrah, de...","- pulau, pesisir cuaca, buruk, mesti, pasrah, ..."
2388,tengah_2081,Pantai Family,-,"pantainya, anak, parkir, rekreasi, biaya, sant...","- pantainya, anak, parkir, rekreasi, biaya, sa..."
2389,tengah_2083,Pantai Biru,-,"namanya airnya, jernih, enak, main, air, baren...","- namanya airnya, jernih, enak, main, air, bar..."
2390,tengah_2084,Pantai Putra Deli,-,"parkir, pungli, laut, biaya, santai, pemandang...","- parkir, pungli, laut, biaya, santai, pemanda..."


In [ ]:
df_cbr_cleaned = df_cbr.drop(columns=['description', 'combined_review_keywords'])
df_cbr_cleaned

,place_id,place_name,combined_text
0,timur_0001,Pantai Natsepa,"Jalur pasir putih dengan pemandian, banana boa..."
1,timur_0002,Pantai Liang,"- tempat wisata, pemandangan, rujak, banana bo..."
2,timur_0004,Pantai Ngurbloat Desa Ngilngof - Kei Islands,Hamparan pasir putih panjang berlatar pohon ke...
3,timur_0005,Pantai Pintu Kota Ambon,Tempat berenang tertutup di samping pantai ber...
4,timur_0006,Pantai Kota Jawa,"- santai, sore, pemandangan, snack, pinggir pa..."
...,...,...,...
2387,tengah_2079,pantai Tanjung kelayang,"- pulau, pesisir cuaca, buruk, mesti, pasrah, ..."
2388,tengah_2081,Pantai Family,"- pantainya, anak, parkir, rekreasi, biaya, sa..."
2389,tengah_2083,Pantai Biru,"- namanya airnya, jernih, enak, main, air, bar..."
2390,tengah_2084,Pantai Putra Deli,"- parkir, pungli, laut, biaya, santai, pemanda..."


In [ ]:
df_cbr_cleaned['combined_text'] = df_cbr_cleaned['combined_text'].str.replace('-', '', regex=False)
print(df_cbr_cleaned[['place_name', 'combined_text']].head())

                                     place_name  \
0                                Pantai Natsepa   
1                                  Pantai Liang   
2  Pantai Ngurbloat Desa Ngilngof - Kei Islands   
3                       Pantai Pintu Kota Ambon   
4                              Pantai Kota Jawa   

                                       combined_text  
0  Jalur pasir putih dengan pemandian, banana boa...  
1   tempat wisata, pemandangan, rujak, banana boa...  
2  Hamparan pasir putih panjang berlatar pohon ke...  
3  Tempat berenang tertutup di samping pantai ber...  
4   santai, sore, pemandangan, snack, pinggir pan...  


In [ ]:
# Simpan ke CSV yang bersih
df_cbr_cleaned[['place_id', 'place_name', 'combined_text']].to_csv('cbr_clean.csv', index=False)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import nltk

In [ ]:
try:
    indonesian_stopwords = stopwords.words('indonesian')
except LookupError:
    nltk.download('stopwords')
    indonesian_stopwords = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
tfidf = TfidfVectorizer(stop_words=indonesian_stopwords)

In [ ]:
tfidf_matrix = tfidf.fit_transform(df_cbr_cleaned['combined_text'])

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


In [ ]:
# User Input
user_input = "wisata, liburan, anginnya, pemandangan, kelapa, nan"

In [ ]:
user_tfidf = tfidf.transform([user_input])

# Hitung kemiripan cosine antara user dan pantai
cosine_sim = cosine_similarity(user_tfidf, tfidf_matrix).flatten()

# Ambil top-N rekomendasi
top_n = 10
top_indices = cosine_sim.argsort()[-top_n:][::-1]

In [ ]:
print("Rekomendasi Pantai Berdasarkan Preferensimu:\n")
for i in top_indices:
    print(f"{df_cbr_cleaned.iloc[i]['place_name']} - Score: {cosine_sim[i]:.3f}")
    print(f"Deskripsi: {df_cbr_cleaned.iloc[i]['combined_text']}\n")

Rekomendasi Pantai Berdasarkan Preferensimu:

Pantai Akasia Indah Guntung - Score: 1.000
Deskripsi:  wisata, liburan, anginnya, pemandangan, kelapa nan

Pantai Teluk Selahang - Score: 0.636
Deskripsi:  kelapa nan

Pantai Exotic - Score: 0.563
Deskripsi:  sejuk, kelapa nan

Pantai Exotic - Score: 0.563
Deskripsi:  sejuk, kelapa nan

Pantai Keramba Apung - Score: 0.551
Deskripsi:  pemandangan nan

Pantai Selayang Pandang - Score: 0.551
Deskripsi:  pemandangan nan

Pantai Botorubuh - Score: 0.523
Deskripsi:  pemandangan, wisata, hati nan

Pantai D'plango Pak Endi - Score: 0.482
Deskripsi:  liburan, ombaknya nan

Pantai Rigaih - Score: 0.453
Deskripsi:  tempat wisata, ombaknya nan

Pantai Peudawa - Score: 0.451
Deskripsi:  wisata, pohon nan



In [ ]:
import pickle

In [ ]:
# Simpan vectorizer dan matriks
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

with open('tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)

In [ ]:
# Load dan pakai kembali
tfidf_vectorizer = pickle.load(open('tfidf_vectorizer.pkl', 'rb'))
tfidf_matrix = pickle.load(open('tfidf_matrix.pkl', 'rb'))

In [ ]:
user_input = "pantai cocok untuk keluarga"
user_vec = tfidf_vectorizer.transform([user_input])

# 4. Hitung similarity
cosine_similarity(user_vec, tfidf_matrix)

array([[0.1165023 , 0.25988669, 0.19490742, ..., 0.26001813, 0.27784711,
        0.13743939]])

In [ ]:
# Lihat kata-kata yang dipelajari
print(tfidf_vectorizer.get_feature_names_out()[:10])

# Lihat matrix bentuknya
print(tfidf_matrix.shape)  # Misal: (100 pantai, 5000 kata unik

['000' '00018' '000rupiah' '07' '081254915467' '081256279849'
 '081325300135' '081339635043' '081584220504' '08217660176t']
(2392, 152732)


# Collaborative